In [1]:
#import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
import numpy as np

In [2]:
teamPath = '/Users/lorneez/projects/sports_predictor/Sports_Model/Data/teamData.csv'
playerPath = '/Users/lorneez/projects/sports_predictor/Sports_Model/Data/playerData.csv'
outcomePath = '/Users/lorneez/projects/sports_predictor/Sports_Model/Data/outcomeData.csv'
teamData = pd.read_csv(teamPath)
playerData = pd.read_csv(playerPath, engine='python')
outcomeData = pd.read_csv(outcomePath)

In [3]:
outcomeData = outcomeData.drop(["Start Time", "Box Score", "Notes"], axis=1)

In [4]:
outcomeData.loc[(outcomeData["OT?"].isna()),'OT?']= 0
outcomeData.loc[(outcomeData["OT?"] == "OT"),'OT?']= 1
outcomeData.loc[(outcomeData["OT?"] == "2OT"),'OT?']= 2
outcomeData.loc[(outcomeData["OT?"] == "3OT"),'OT?']= 3
outcomeData.loc[(outcomeData["OT?"] == "4OT"),'OT?']= 4

In [5]:
def redefineTeamNames(dataframe):
    dataframe = dataframe.replace('Seattle SuperSonics', 'Oklahoma City Thunder', regex=True)
    
    dataframe = dataframe.replace('New Orleans/Oklahoma City Hornets', 'New Orleans Pelicans', regex=True)

    dataframe = dataframe.replace('New Orleans Hornets', 'New Orleans Pelicans', regex=True)
    
    dataframe = dataframe.replace('Charlotte Bobcats', 'Charlotte Hornets', regex=True)
    
    dataframe = dataframe.replace('New Jersey Nets', 'Brooklyn Nets', regex=True)
    
#     dataframe = dataframe.replace('*', '', regex=True)

    return dataframe

outcomeData = redefineTeamNames(outcomeData)

In [6]:
def homeWin(dataframe):
    if dataframe['Winner'] == dataframe['Home']:
        return True
    else:
        return False

outcomeData['HomeWin'] = outcomeData.apply(homeWin, axis=1)

In [7]:
encoding = LabelEncoder()
encoding.fit(outcomeData['Home'].values)
outcomeData['Home'] = encoding.transform(outcomeData['Home'].values)
outcomeData['Visitor'] = encoding.transform(outcomeData['Visitor'].values)

In [13]:
featurePath = '/Users/lorneez/projects/sports_predictor/Sports_Model/Data/feature_data.csv'
X = pd.read_csv(featurePath)
y = outcomeData["HomeWin"]

print(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

       Unnamed: 0                                FGA_PG_home  \
0               0   23    78.3\nName: FGA_PG, dtype: float64   
1               1    2    81.6\nName: FGA_PG, dtype: float64   
2               2   11    80.8\nName: FGA_PG, dtype: float64   
3               3   16    81.5\nName: FGA_PG, dtype: float64   
4               4    8    81.5\nName: FGA_PG, dtype: float64   
...           ...                                        ...   
20429       20429  478    88.2\nName: FGA_PG, dtype: float64   
20430       20430  467    87.7\nName: FGA_PG, dtype: float64   
20431       20431  454    90.6\nName: FGA_PG, dtype: float64   
20432       20432  460    84.4\nName: FGA_PG, dtype: float64   
20433       20433  450    90.0\nName: FGA_PG, dtype: float64   

                                  FGA_PG_visitor  \
0       19    78.3\nName: FGA_PG, dtype: float64   
1        1    85.1\nName: FGA_PG, dtype: float64   
2        7    80.7\nName: FGA_PG, dtype: float64   
3       24    75.2\nNam

In [11]:
from keras import Sequential
from keras.layers import Dense

classifier = Sequential()
#First Hidden Layer
classifier.add(Dense(4, activation='relu', kernel_initializer='random_normal', input_dim=29))
#Second  Hidden Layer
classifier.add(Dense(4, activation='relu', kernel_initializer='random_normal'))
#Output Layer
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))

#Compiling the neural network
classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])

In [12]:
#Fitting the data to the training dataset
classifier.fit(X_train,y_train, batch_size=10, epochs=100)

Epoch 1/100


ValueError: could not convert string to float: '155    78.0\nName: FGA_PG, dtype: float64'

In [ ]:
eval_model=classifier.evaluate(X_train, y_train)
eval_model

In [ ]:
y_pred=classifier.predict(X_test)
y_pred =(y_pred>0.5)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

In [ ]:
(1277+2931)/(1277+1241+682+2931)